In [ ]:
!git clone https://github.com/CASIA-IVA-Lab/FastSAM.git

Cloning into 'FastSAM'...
remote: Enumerating objects: 1221, done.
remote: Counting objects: 100% (386/386), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 1221 (delta 334), reused 305 (delta 305), pack-reused 835
Receiving objects: 100% (1221/1221), 72.46 MiB | 30.21 MiB/s, done.
Resolving deltas: 100% (487/487), done.


In [ ]:
!wget https://huggingface.co/spaces/An-619/FastSAM/resolve/main/weights/FastSAM.pt

--2024-06-12 04:01:33--  https://huggingface.co/spaces/An-619/FastSAM/resolve/main/weights/FastSAM.pt
Resolving huggingface.co (huggingface.co)... 18.172.134.4, 18.172.134.88, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/71/c3/71c3ec5a83f3fc374665cec9240f614262e29088fca1a18732571e13c7084f1b/c0be4e7ddbe4c15333d15a859c676d053c486d0a746a3be6a7a9790d52a9b6d7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27FastSAM.pt%3B+filename%3D%22FastSAM.pt%22%3B&Expires=1718424093&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxODQyNDA5M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83MS9jMy83MWMzZWM1YTgzZjNmYzM3NDY2NWNlYzkyNDBmNjE0MjYyZTI5MDg4ZmNhMWExODczMjU3MWUxM2M3MDg0ZjFiL2MwYmU0ZTdkZGJlNGMxNTMzM2QxNWE4NTljNjc2ZDA1M2M0ODZkMGE3NDZhM2JlNmE3YTk3OTBkNTJhOWI2ZDc%7EcmVzcG9uc2UtY29

In [ ]:
!pip install -r FastSAM/requirements.txt
!pip install git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.7/611.7 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py

**Main code goes below**

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import zipfile
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**MAIN PROGRAM**

In [ ]:
# DEFINE CURRENT LEVEL [0-4]

current_level = 4

# number of images to process

num_img_to_process = 1000

In [ ]:

# Define paths
zip_file_path = f'/content/drive/MyDrive/Colab Notebooks/Thesis_UG_Buet/xray_images/{current_level}/{current_level}.zip'
extraction_path = '/content/images/'  # Destination folder for extracted images

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)


In [ ]:

image_folder = f'/content/images/{current_level}'

# Get a list of all image files in the directory
image_files = os.listdir(image_folder)


**Will take a lot of time**

In [ ]:

# Loop over each image file and generate segmented image

total_images = len(image_files)
if(total_images>1000):
  num_img_to_process = 1000
else:
  num_img_to_process = total_images

for idx, image_name in enumerate(image_files[0:num_img_to_process]):
    if image_name.endswith(('.jpg', '.jpeg', '.png')):  # Ensure it's an image file
        # Load and preprocess input image
        print(f"Processing {idx + 1}/{total_images} images: {image_name}")
        image_path = os.path.join(image_folder, image_name)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


        # Run inference on the input image using FastSAM
        !python FastSAM/Inference.py --model_path FastSAM.pt --img_path $image_path --imgsz 224



# immediately zip output folder and download
# Path to the output folder
output_folder = '/content/output/'

# List all files in the output folder
files = os.listdir(output_folder)

# Rename each file by adding 's_' before the filename
for filename in files:
    # Construct the old and new paths
    old_path = os.path.join(output_folder, filename)
    new_filename = 's_' + filename
    new_path = os.path.join(output_folder, new_filename)

    # Rename the file
    os.rename(old_path, new_path)

# Zip the Output Folder
!zip -r output.zip output

# Download the Zip File
from google.colab import files
files.download('output.zip')


Processing 1/173 images: 9746416R.png
2024-06-12 04:05:37.102721: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 04:05:37.102799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 04:05:37.104926: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

0: 224x224 1 object, 1017.6ms
Speed: 4.9ms preprocess, 1017.6ms inference, 36.4ms postprocess per image at shape (1, 3, 224, 224)
Processing 2/173 images: 9716393L.png
2024-06-12 04:05:55.491984: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory f

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Delete every image from output**


**Delete output**

In [ ]:

# List all files in the image directory
image_files = os.listdir(image_folder)

# Loop through each file and delete it if it's a file (not a directory)
for file_name in image_files:
    file_path = os.path.join(image_folder, file_name)
    if os.path.isfile(file_path):
        os.remove(file_path)
shutil.rmtree(image_folder)


# Define the path to the output directory
output_folder = '/content/output/'
output_files = os.listdir(output_folder)

# Delete all files in the output directory
for file_name in output_files:
    file_path = os.path.join(output_folder, file_name)
    if os.path.isfile(file_path):
        os.remove(file_path)
shutil.rmtree(output_folder)

# Delete the output.zip file if it exists
zip_file_path = 'output.zip'
if os.path.exists(zip_file_path):
    os.remove(zip_file_path)


# **level 2, 3, 4 at a time**

In [ ]:
'''
for current_level in range(2, 5):
  # Define paths
  zip_file_path = f'/content/drive/MyDrive/Colab Notebooks/Thesis_UG_Buet/xray_images/{current_level}/{current_level}.zip'
  extraction_path = '/content/images/'  # Destination folder for extracted images

  # Unzip the file
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall(extraction_path)
  image_folder = f'/content/images/{current_level}'

  # Get a list of all image files in the directory
  image_files = os.listdir(image_folder)

  # Loop over each image file and generate segmented image

  total_images = len(image_files)
  if(total_images>1000):
    num_img_to_process = 1000
  else:
    num_img_to_process = total_images
  for idx, image_name in enumerate(image_files[0:num_img_to_process]):
      if image_name.endswith(('.jpg', '.jpeg', '.png')):  # Ensure it's an image file
          # Load and preprocess input image
          print(f"Processing {idx + 1}/{total_images} images level-{current_level}: {image_name}")
          image_path = os.path.join(image_folder, image_name)
          image = cv2.imread(image_path)
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


          # Run inference on the input image using FastSAM
          !python FastSAM/Inference.py --model_path FastSAM.pt --img_path $image_path --imgsz 224



  # immediately zip output folder and download
  # Path to the output folder
  output_folder = '/content/output/'

  # List all files in the output folder
  files = os.listdir(output_folder)

  # Rename each file by adding 's_' before the filename
  for filename in files:
      # Construct the old and new paths
      old_path = os.path.join(output_folder, filename)
      new_filename = 's_' + filename
      new_path = os.path.join(output_folder, new_filename)

      # Rename the file
      os.rename(old_path, new_path)

  # Zip the Output Folder
  !zip -r output.zip output

  # Download the Zip File
  from google.colab import files
  files.download('output.zip')

  # List all files in the image directory
  image_files = os.listdir(image_folder)

  # Loop through each file and delete it if it's a file (not a directory)
  for file_name in image_files:
      file_path = os.path.join(image_folder, file_name)
      if os.path.isfile(file_path):
          os.remove(file_path)
  shutil.rmtree(image_folder)


  # Define the path to the output directory
  output_files = os.listdir(output_folder)

  # Delete all files in the output directory
  for file_name in output_files:
      file_path = os.path.join(output_folder, file_name)
      if os.path.isfile(file_path):
          os.remove(file_path)
  shutil.rmtree(output_folder)

  # Delete the output.zip file if it exists
  zip_file_path = 'output.zip'
  if os.path.exists(zip_file_path):
      os.remove(zip_file_path)
'''
